In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iyzico-datathon/sample_submission.csv
/kaggle/input/iyzico-datathon/train.csv


## AÇIKLAMALAR

# Train ve Test Veri Setleri

* merchant_id Maskelenmiş iş yeri ID'si

* month_id İşlemin yapıldığı ay (YYYYMM formatında)

* merchant_source İş yerinin iyzico’ya katıldığı kaynak

* settlement_period İş yerinin hak edişini alış sıklığı

* working_type İş yerinin tipini gösterir

* mcc_id İş yerinin satış yaptığı kategori bilgisini gösterir

* merchant_segment İş yerinin iyzico içerisinde bulunduğu segmenti gösterir

* net_payment_count İş yerinin ilgili ay içerisinde geçirdiği net (ödeme - iptal - iade) işlem sayısıdır

# Submission Dosyası

* id Tahminlenmesi beklenen ay ve maskelenmiş müşteri ID'si (Örn: 202312merchant_36004)

* net_payment_count İş yerinin ilgili ay geçirdiği işlem sayısı

# Dış Veri Kullanımı

Dönemsel enflasyon verisi, maaş ödenme günleri, (SGK ve devlet memurlarının maaş günleri vb…), resmi ve dini bayramlara ilişkin veriler, döviz kurları, mevsime göre ülkemizde görülen ortalama sıcaklık verileri gibi modelin kurulumunda size faydalı olabileceğini düşündüğünüz her türlü dış veriyi kullanabilirsiniz.


In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
sample_submission = pd.read_csv("/kaggle/input/iyzico-datathon/sample_submission.csv")
base_train = pd.read_csv("/kaggle/input/iyzico-datathon/train.csv")

In [5]:
sample_submission

,id,net_payment_count
0,202311merchant_36004,0
1,202312merchant_36004,0
2,202310merchant_36004,0
3,202311merchant_23099,0
4,202312merchant_23099,0
...,...,...
78175,202312merchant_35969,0
78176,202310merchant_35969,0
78177,202311merchant_8429,0
78178,202312merchant_8429,0


In [6]:
base_train

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,merchant_43992,202307,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,15106
1,merchant_43992,202301,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16918
2,merchant_43992,202305,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,13452
3,merchant_43992,202308,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16787
4,merchant_43992,202302,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,12428
...,...,...,...,...,...,...,...,...
291137,merchant_67083,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_169,Segment - 4,4
291138,merchant_4263,202309,Merchant Source - 2,Settlement Period - 1,Working Type - 6,mcc_73,Segment - 4,4
291139,merchant_11346,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_169,Segment - 4,3
291140,merchant_21397,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_168,Segment - 4,3


In [7]:
base_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291142 entries, 0 to 291141
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   merchant_id           291142 non-null  object
 1   month_id              291142 non-null  int64 
 2   merchant_source_name  291142 non-null  object
 3   settlement_period     291142 non-null  object
 4   working_type          291142 non-null  object
 5   mcc_id                291142 non-null  object
 6   merchant_segment      291142 non-null  object
 7   net_payment_count     291142 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 17.8+ MB


In [8]:
base_train.describe()

,month_id,net_payment_count
count,291142.000000,2.911420e+05
mean,202159.680860,4.153842e+02
std,104.214948,1.082067e+04
min,202001.000000,-1.433000e+03
25%,202103.000000,4.000000e+00
50%,202201.000000,6.000000e+00
75%,202211.000000,1.400000e+01
max,202309.000000,1.160429e+06


In [9]:
print(f"Unique values for merchant id {base_train['merchant_id'].nunique()}")
print(f"Unique values for month id {base_train['month_id'].nunique()}")
print(f"Unique values for merchant_source_name {base_train['merchant_source_name'].nunique()}")
print(f"Unique values for settlement_period {base_train['settlement_period'].nunique()}")
print(f"Unique values for working_type {base_train['working_type'].nunique()}")
print(f"Unique values for mcc_id {base_train['mcc_id'].nunique()}")
print(f"Unique values for merchant_segment {base_train['merchant_segment'].nunique()}")
print(f"Unique values for net_payment_count {base_train['net_payment_count'].nunique()}")





Unique values for merchant id 26060
Unique values for month id 45
Unique values for merchant_source_name 3
Unique values for settlement_period 3
Unique values for working_type 6
Unique values for mcc_id 172
Unique values for merchant_segment 4
Unique values for net_payment_count 5039


In [10]:
# Example preprocessing (modify as needed)
cluster_data = base_train.copy() # Remove unnecessary columns
cluster_data = pd.get_dummies(cluster_data, columns= ["working_type", "merchant_segment", "merchant_source_name", "settlement_period"])  # One-hot encode categorical variables if necessary
cluster_data

,merchant_id,month_id,mcc_id,net_payment_count,working_type_Working Type - 1,working_type_Working Type - 2,working_type_Working Type - 3,working_type_Working Type - 4,working_type_Working Type - 5,working_type_Working Type - 6,merchant_segment_Segment - 1,merchant_segment_Segment - 2,merchant_segment_Segment - 3,merchant_segment_Segment - 4,merchant_source_name_Merchant Source - 1,merchant_source_name_Merchant Source - 2,merchant_source_name_Merchant Source - 3,settlement_period_Settlement Period - 1,settlement_period_Settlement Period - 2,settlement_period_Settlement Period - 3
0,merchant_43992,202307,mcc_197,15106,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True
1,merchant_43992,202301,mcc_197,16918,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True
2,merchant_43992,202305,mcc_197,13452,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True
3,merchant_43992,202308,mcc_197,16787,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True
4,merchant_43992,202302,mcc_197,12428,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291137,merchant_67083,202309,mcc_169,4,False,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False
291138,merchant_4263,202309,mcc_73,4,False,False,False,False,False,True,False,False,False,True,False,True,False,True,False,False
291139,merchant_11346,202309,mcc_169,3,False,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False
291140,merchant_21397,202309,mcc_168,3,False,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False


In [11]:
for column in base_train.columns:
    print(base_train[column].value_counts())

merchant_id
merchant_43992    45
merchant_19380    45
merchant_1100     45
merchant_17200    45
merchant_65293    45
                  ..
merchant_3063      1
merchant_35756     1
merchant_50399     1
merchant_56272     1
merchant_66895     1
Name: count, Length: 26060, dtype: int64
month_id
202112    7413
202303    7367
202203    7332
202305    7327
202308    7315
202309    7287
202204    7244
202307    7217
202201    7210
202301    7198
202304    7191
202212    7153
202111    7129
202211    7086
202210    7073
202205    7072
202306    7065
202202    7064
202105    7045
202206    7040
202109    7021
202110    7018
202106    7004
202209    6967
202103    6883
202104    6878
202208    6876
202302    6804
202108    6771
202207    6648
202102    6607
202107    6490
202101    6437
202012    6352
202011    5887
202010    5654
202009    5489
202008    5224
202007    4990
202006    4920
202005    4707
202004    4120
202003    3951
202002    3867
202001    3749
Name: count, dtype: int64
mercha

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming 'net_payment_count' is your target variable
target_variable = "net_payment_count"

# Selecting features (excluding the target variable)
cluster_data = cluster_data.drop(columns=[target_variable])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cluster_data, cluster_data[target_variable], test_size=0.2, random_state=42)

# Create a Random Forest Classifier model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
classification_report_result = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_report_result)


KeyError: 'net_payment_count'

In [ ]:
# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importances
print("Feature Importances:")
print(feature_importance_df)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.show()